In [316]:
import numpy as np
import re
import pickle
import h5py
from random import sample
import tensorflow as tf
import tensorflow_probability as tfp
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
from keras.layers import LSTM, Embedding
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from feature_funcs import *
from keras.utils import to_categorical

## Prepare Training Data

In [2]:
# Create dictionary
with open("../data/Jigs.txt") as my_file:
    abc_text = my_file.read()

# Cut out unnecessary backslashes
abc_text = re.sub('\\\\+\n', '\n', abc_text)

# Find starting index of the data we care about
start_ind = abc_text.find("X:")
abc_text = abc_text[start_ind:]

# Encode data
num_to_char, char_to_num = create_dictionaries(abc_text)
vocab_length = len(num_to_char)

### Separate Songs

In [3]:
songs = re.findall(r'(T:(?:.+\n)+\n+)X:', abc_text)

song_texts = []

for song in songs:
    song_text = re.search(r'(P:.+|K:.+)', song, re.DOTALL)
    song_texts.append(song_text[0])
    
# Get aggregate statistics about song texts
print("Min text length: ", min([len(text) for text in song_texts]))
print("Max text length: ", max([len(text) for text in song_texts]))
print("Average text length: ", np.mean([len(text) for text in song_texts]))

# Make sure all song texts are the same length
song_texts = [text[:170] for text in song_texts]

Min text length:  177
Max text length:  1200
Average text length:  301.5929203539823


### Extract Metadata elements

In [473]:
# Extract titles
titles = []

for song in songs:
    title = re.search(r'(T:.+\n% Nottingham Music Database\n)', song).group(1)
    titles.append(title)
    
# Pad titles to same length
max_length = max(len(title) for title in titles)
titles = [title.ljust(max_length) for title in titles]


# Extract authors
authors = []

for song in songs:
    author = re.search(r'\n(S:.+\n)', song).group(1)
    authors.append(author)
    
# Pad authors to same length
max_length = max(len(author) for author in authors)
authors = [author.ljust(max_length) for author in authors]


# Extract meters
meters = []

for song in songs:
    meter = re.search(r'(M:\d/\d\n)', song).group(1)
    meters.append(meter)

    
# Pad meters to same length
max_length = max(len(meter) for meter in meters)
meters = [meter.ljust(max_length) for meter in meters]


### Encode data to numeric format

In [477]:
text_nums_list = encoder2(song_texts, char_to_num)

title_nums = encoder2(titles, char_to_num)
author_nums = encoder2(authors, char_to_num)
meter_nums = encoder2(meters, char_to_num)

In [479]:
meter_nums

array([[58., 75., 10., 49., 10.,  2.],
       [58., 75., 17., 49., 80.,  2.],
       [58., 75., 17., 49., 80.,  2.],
       ...,
       [58., 75., 17., 49., 80.,  2.],
       [58., 75., 17., 49., 80.,  2.],
       [58., 75., 17., 49., 80.,  2.]])

### Option 2: Use Pickled data where songs are not separated (data already encoded to numeric)

In [ ]:
# Open pickled training data so you don't have to re-run create_training
x_file_pickle = open('../data/x_train_pickle.obj', 'rb')
y_file_pickle = open('../data/y_train_pickle.obj', 'rb')

x_train = pickle.load(x_file_pickle)
y_train = pickle.load(y_file_pickle)

x_file_pickle.close()
y_file_pickle.close()

vocab_length = x_train.shape[2]
vocab_length

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
print(vocab_length)

## Build & Compile LSTM (Model 1)

Sources: <br />
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  <br />
https://github.com/aamini/introtodeeplearning/blob/master/lab1/Part2_Music_Generation.ipynb  <br /> https://medium.com/datadriveninvestor/music-generation-using-deep-learning-85010fb982e2 
<br /> https://keras.io/examples/lstm_text_generation/

In [104]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(512))
model.add(layers.Dense(vocab_length, activation='softmax'))

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 166, 128)          111104    
_________________________________________________________________
lstm_13 (LSTM)               (None, 166, 256)          394240    
_________________________________________________________________
lstm_14 (LSTM)               (None, 512)               1574912   
_________________________________________________________________
dense_4 (Dense)              (None, 88)                45144     
Total params: 2,125,400
Trainable params: 2,125,400
Non-trainable params: 0
_________________________________________________________________


In [105]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### Train RNN
Epoch: When the Neural Network sees all of the training data <br />
Batch: Subset of the data <br />
i.e. If you have 1000 data points, your batch size is 500 and you want 1 epoch, then the NN will do 2 iterations.

In [106]:
# Use checkpoints to save training weights before the model finishes training
# Using this file path, the model checkpoints will be saved with the epoch number 
# and the validation loss in the filename.
weights_filepath = "weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

# Fit model
model.fit(x_train, y_train, epochs=1, batch_size=400, callbacks=callbacks_list)

Train on 237 samples
237/237 [==============================] - 10s 42ms/sample - loss: 4.4778


In [107]:
# Continue training model
model.load_weights("weights.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Fit model
model.fit(x_train, y_train, epochs=1, batch_size=200, callbacks=callbacks_list)

Train on 237 samples
237/237 [==============================] - 12s 51ms/sample - loss: 4.3406


In [ ]:
# Open pickled test data so you don't have to re-run create_training
x_file_pickle = open('../data/x_test_pickle.obj', 'rb')
y_file_pickle = open('../data/y_test_pickle.obj', 'rb')

x_test = pickle.load(x_file_pickle)
y_test = pickle.load(y_file_pickle)

x_file_pickle.close()
y_file_pickle.close()

In [111]:
predictions = model.predict(x_test)
predictions

array([[0.00064478, 0.00090162, 0.00271225, ..., 0.01233923, 0.001392  ,
        0.00071112],
       [0.00063128, 0.0008802 , 0.00265699, ..., 0.01220111, 0.00136681,
        0.0006962 ],
       [0.00064796, 0.0009045 , 0.00271147, ..., 0.0124012 , 0.00139878,
        0.00071476],
       ...,
       [0.00064573, 0.0009006 , 0.00270226, ..., 0.01237478, 0.00138951,
        0.00071131],
       [0.00065404, 0.00091246, 0.00271773, ..., 0.01246372, 0.00140417,
        0.00071928],
       [0.00062923, 0.00087897, 0.00265093, ..., 0.01220588, 0.00136166,
        0.00069508]], dtype=float32)

In [112]:
text_predictions = decoder(predictions, num_to_char)
print(text_predictions)

## Build & Compile LSTM (Model 2) 

Sources:<br /> https://benjamintseng.com/portfolio/nlp-pubmed-data-using-tensorflow-and-keras/ <br />
https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [462]:
def LSTM_Builder(song_text_nums, metadata_nums, str_length, vocab_size):
    # Prepare data
    if str_length+1 > len(song_text_nums[0]):
        return "Your string length is too long for the data."

    # The x_values begin at the start of each song text and are str_length characters long;
    # Concatenate these with the metadata at the beginning
    # The y_values are one character after the end of the x_values
    x_data = np.concatenate((metadata_nums[0], song_text_nums[0][0:str_length]))
    y_data = [song_text_nums[0][str_length]]
    
    for ind, song in enumerate(song_text_nums[1:], start=1):
        x_concat = np.concatenate((metadata_nums[i], np.array(song[0:str_length])))
        x_data = np.vstack((x_data, x_concat))
        y_data.append(song[str_length])

    # Convert x and y data to tensors
    x_train = to_categorical(x_data, num_classes=vocab_size)
    y_train = to_categorical(y_data, num_classes=vocab_size)
    
    # Build NN
    model = tf.keras.Sequential()
    model.add(layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(vocab_length, activation='softmax'))
    
    return x_train, y_train, model


In [463]:
x_train, y_train, mod2 = LSTM_Builder(text_nums_list[:train_ind], meter_nums, 160, vocab_length)
mod2.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 166, 128)          111104    
_________________________________________________________________
dropout_6 (Dropout)          (None, 166, 128)          0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 166, 256)          394240    
_________________________________________________________________
dropout_7 (Dropout)          (None, 166, 256)          0         
_________________________________________________________________
lstm_48 (LSTM)               (None, 512)               1574912   
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 88)              

In [464]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
mod2.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [465]:
# Use checkpoints to save training weights before the model finishes training
# Using this file path, the model checkpoints will be saved with the epoch number 
# and the validation loss in the filename.
weights_filepath = "mod2_weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

In [466]:
# Fit model
mod2.fit(x_train, y_train, epochs=1000, batch_size=100, callbacks=callbacks_list)

Train on 237 samples
Epoch 1/1000
237/237 [==============================] - 26s 108ms/sample - loss: 4.5451
Epoch 2/1000
237/237 [==============================] - 16s 68ms/sample - loss: 3.5298
Epoch 3/1000
237/237 [==============================] - 14s 59ms/sample - loss: 3.1888
Epoch 4/1000
237/237 [==============================] - 14s 58ms/sample - loss: 3.1248
Epoch 5/1000
237/237 [==============================] - 13s 53ms/sample - loss: 3.0493
Epoch 6/1000
237/237 [==============================] - 18s 76ms/sample - loss: 3.0537
Epoch 7/1000
237/237 [==============================] - 14s 61ms/sample - loss: 3.0598
Epoch 8/1000
237/237 [==============================] - 12s 51ms/sample - loss: 3.0906
Epoch 9/1000
237/237 [==============================] - 13s 56ms/sample - loss: 3.0310
Epoch 10/1000
237/237 [==============================] - 13s 55ms/sample - loss: 3.0292
Epoch 11/1000
237/237 [==============================] - 12s 50ms/sample - loss: 3.1112
Epoch 12/1000
237/2

Epoch 94/1000
237/237 [==============================] - 13s 54ms/sample - loss: 0.0548
Epoch 95/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0429
Epoch 96/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0357
Epoch 97/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0310
Epoch 98/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0240
Epoch 99/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0212
Epoch 100/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0177
Epoch 101/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0160
Epoch 102/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0142
Epoch 103/1000
237/237 [==============================] - 12s 52ms/sample - loss: 0.0123
Epoch 104/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0100
Epoch 105/1000
237/237 [===

237/237 [==============================] - 12s 52ms/sample - loss: 1.0169e-04
Epoch 186/1000
237/237 [==============================] - 12s 52ms/sample - loss: 8.5527e-05
Epoch 187/1000
237/237 [==============================] - 12s 52ms/sample - loss: 7.7942e-05
Epoch 188/1000
237/237 [==============================] - 12s 52ms/sample - loss: 6.4759e-05
Epoch 189/1000
237/237 [==============================] - 12s 52ms/sample - loss: 6.0071e-05
Epoch 190/1000
237/237 [==============================] - 12s 52ms/sample - loss: 4.9220e-05
Epoch 191/1000
237/237 [==============================] - 12s 52ms/sample - loss: 4.3534e-05
Epoch 192/1000
237/237 [==============================] - 12s 52ms/sample - loss: 3.6079e-05
Epoch 193/1000
237/237 [==============================] - 12s 52ms/sample - loss: 3.3304e-05
Epoch 194/1000
237/237 [==============================] - 12s 52ms/sample - loss: 2.8769e-05
Epoch 195/1000
237/237 [==============================] - 12s 52ms/sample - loss: 2.4

Epoch 275/1000
237/237 [==============================] - 12s 52ms/sample - loss: 5.5671e-04
Epoch 276/1000
237/237 [==============================] - 12s 52ms/sample - loss: 4.6947e-04
Epoch 277/1000
237/237 [==============================] - 12s 51ms/sample - loss: 4.4762e-04
Epoch 278/1000
237/237 [==============================] - 12s 52ms/sample - loss: 3.6128e-04
Epoch 279/1000
237/237 [==============================] - 12s 52ms/sample - loss: 3.5084e-04
Epoch 280/1000
237/237 [==============================] - 12s 52ms/sample - loss: 3.0193e-04
Epoch 281/1000
237/237 [==============================] - 12s 51ms/sample - loss: 2.7390e-04
Epoch 282/1000
237/237 [==============================] - 12s 52ms/sample - loss: 2.2235e-04
Epoch 283/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.9697e-04
Epoch 284/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.7620e-04
Epoch 285/1000
237/237 [==============================] - 12s 51ms/sam

237/237 [==============================] - 12s 51ms/sample - loss: 0.0019
Epoch 365/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0014
Epoch 366/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0012
Epoch 367/1000
237/237 [==============================] - 12s 52ms/sample - loss: 0.0011
Epoch 368/1000
237/237 [==============================] - 12s 51ms/sample - loss: 0.0010
Epoch 369/1000
237/237 [==============================] - 12s 52ms/sample - loss: 9.5823e-04
Epoch 370/1000
237/237 [==============================] - 12s 52ms/sample - loss: 7.3383e-04
Epoch 371/1000
237/237 [==============================] - 12s 52ms/sample - loss: 7.0775e-04
Epoch 372/1000
237/237 [==============================] - 12s 52ms/sample - loss: 5.9935e-04
Epoch 373/1000
237/237 [==============================] - 12s 52ms/sample - loss: 4.9508e-04
Epoch 374/1000
237/237 [==============================] - 12s 51ms/sample - loss: 5.5868e-04
Epoch 375/10

...

237/237 [==============================] - 12s 52ms/sample - loss: 1.9164e-07
Epoch 629/1000
237/237 [==============================] - 16s 67ms/sample - loss: 1.4838e-07
Epoch 630/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.2776e-07
Epoch 631/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.2877e-07
Epoch 632/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.2927e-07
Epoch 633/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.3178e-07
Epoch 634/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.1871e-07
Epoch 635/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.1066e-07
Epoch 636/1000
237/237 [==============================] - 12s 52ms/sample - loss: 9.3557e-08
Epoch 637/1000
237/237 [==============================] - 12s 51ms/sample - loss: 8.2994e-08
Epoch 638/1000
237/237 [==============================] - 12s 52ms/sample - loss: 8.5

237/237 [==============================] - 13s 54ms/sample - loss: 2.2132e-08
Epoch 717/1000
237/237 [==============================] - 12s 53ms/sample - loss: 1.8611e-08
Epoch 718/1000
237/237 [==============================] - 12s 51ms/sample - loss: 2.4144e-08
Epoch 719/1000
237/237 [==============================] - 12s 52ms/sample - loss: 2.5653e-08
Epoch 720/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.9114e-08
Epoch 721/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.7605e-08
Epoch 722/1000
237/237 [==============================] - 12s 52ms/sample - loss: 2.7665e-08
Epoch 723/1000
237/237 [==============================] - 12s 51ms/sample - loss: 2.9174e-08
Epoch 724/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.8611e-08
Epoch 725/1000
237/237 [==============================] - 12s 51ms/sample - loss: 2.1629e-08
Epoch 726/1000
237/237 [==============================] - 12s 51ms/sample - loss: 2.2

237/237 [==============================] - 12s 51ms/sample - loss: 1.3078e-08
Epoch 805/1000
237/237 [==============================] - 12s 51ms/sample - loss: 9.0539e-09
Epoch 806/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.4084e-08
Epoch 807/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.0563e-08
Epoch 808/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.1569e-08
Epoch 809/1000
237/237 [==============================] - 12s 51ms/sample - loss: 8.0479e-09
Epoch 810/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.0563e-08
Epoch 811/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.2575e-08
Epoch 812/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.7102e-08
Epoch 813/1000
237/237 [==============================] - 12s 52ms/sample - loss: 1.5090e-08
Epoch 814/1000
237/237 [==============================] - 12s 52ms/sample - loss: 9.0

237/237 [==============================] - 12s 51ms/sample - loss: 8.0479e-09
Epoch 893/1000
237/237 [==============================] - 12s 51ms/sample - loss: 9.5569e-09
Epoch 894/1000
237/237 [==============================] - 12s 51ms/sample - loss: 8.0479e-09
Epoch 895/1000
237/237 [==============================] - 12s 52ms/sample - loss: 8.5509e-09
Epoch 896/1000
237/237 [==============================] - 12s 52ms/sample - loss: 7.5449e-09
Epoch 897/1000
237/237 [==============================] - 12s 51ms/sample - loss: 7.0419e-09
Epoch 898/1000
237/237 [==============================] - 12s 51ms/sample - loss: 7.5449e-09
Epoch 899/1000
237/237 [==============================] - 12s 51ms/sample - loss: 7.0419e-09
Epoch 900/1000
237/237 [==============================] - 12s 51ms/sample - loss: 1.0060e-08
Epoch 901/1000
237/237 [==============================] - 13s 53ms/sample - loss: 9.5569e-09
Epoch 902/1000
237/237 [==============================] - 15s 64ms/sample - loss: 9.0

237/237 [==============================] - 12s 51ms/sample - loss: 3.5209e-09
Epoch 981/1000
237/237 [==============================] - 12s 51ms/sample - loss: 6.5389e-09
Epoch 982/1000
237/237 [==============================] - 12s 51ms/sample - loss: 7.5449e-09
Epoch 983/1000
237/237 [==============================] - 12s 51ms/sample - loss: 6.0359e-09
Epoch 984/1000
237/237 [==============================] - 12s 51ms/sample - loss: 9.0539e-09
Epoch 985/1000
237/237 [==============================] - 12s 51ms/sample - loss: 6.0359e-09
Epoch 986/1000
237/237 [==============================] - 12s 51ms/sample - loss: 5.0299e-09
Epoch 987/1000
237/237 [==============================] - 12s 51ms/sample - loss: 5.5329e-09
Epoch 988/1000
237/237 [==============================] - 12s 51ms/sample - loss: 6.5389e-09
Epoch 989/1000
237/237 [==============================] - 12s 51ms/sample - loss: 6.5389e-09
Epoch 990/1000
237/237 [==============================] - 12s 51ms/sample - loss: 4.5

In [467]:
# Get testing data
test_nums = text_nums_list[train_ind:]
meter_test = meter_nums[train_ind:]
str_length = 160

x_test = np.concatenate((meter_test[0], test_nums[0][0:str_length]))
y_test = [test_nums[0][str_length]]

# The x_values begin at the start of each song text and are str_length characters long;
# Concatenate these with the metadata at the beginning
# The y_values are one character after the end of the x_values    
for ind, song in enumerate(test_nums[1:], start=1):
    x_concat = np.concatenate((meter_test[ind], song[0:str_length]))
    x_test = np.vstack((x_test, x_concat))
    y_test.append(song[str_length])

# Convert x and y data to tensors
x_test = to_categorical(x_test, num_classes=vocab_length)
y_test = to_categorical(y_test, num_classes=vocab_length)

In [468]:
x_test.shape, y_test.shape

((102, 166, 88), (102, 88))

### Generate Predictions

In [469]:
def decoder(binary_matrix, dictionary):
    '''Convert numeric list a text string.'''
    text_list = []
    
    for row in binary_matrix:
        max_ind = np.argmax(row)
        text_list.append(dictionary[max_ind])
    
    return "".join(text_list)

In [470]:
def decoder2(pred_matrix, dictionary):
    '''Convert numeric list a text string.'''
    my_session = tf.Session()
    
    text_list = []
    
    samples = tf.random.categorical(pred_matrix, num_samples=1)
    samples = tf.squeeze(samples, axis=-1)
    vals = sess.run(samples)
    
    text_pred = "".join([dictionary[i] for i in vals])
        
    return text_pred


In [444]:
decoder2(predictions, num_to_char)

'cU}y0:jhHBXB~5Y( &8-nWr MKM3%AebBXanKbA/OIy(E)_)a#Yc|"E!a\\+\\kFhRVQ+4KL(S~K-#}\'(L_]l0}][>QMLpOih\'>YwJkv'

In [471]:
starter = x_test[0]
len_starter = len(x_test[0])

x_vals = np.reshape(starter, (1, starter.shape[0], starter.shape[1]))
result = decoder(starter, num_to_char)
seq_len = 200

for i in range(1,seq_len):
    prediction = mod2.predict(x_vals)
    starter = np.vstack((starter, prediction))
    starter = starter[1:1+len_starter,]
    
    x_vals = np.reshape(starter, (1, starter.shape[0], starter.shape[1]))
    
    text_prediction = decoder2(prediction, num_to_char)
    result += text_prediction

In [436]:
sampled_indices = tf.random.categorical(predictions, num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1)
sess.run(sampled_indices)

array([53, 14, 78, 87, 22,  2, 27,  1, 60, 40, 16, 69, 42, 49, 20, 54, 65,
       26, 78, 50, 54, 81, 85, 12, 62, 38, 69, 71, 40, 85, 48, 32, 13, 58,
       74, 20, 85, 16, 24, 37, 68, 69, 50, 58, 84, 37, 62, 30, 87, 77, 31,
        9, 51, 71,  6, 10,  0, 15, 34, 55, 11, 48,  1, 31, 59, 69,  1, 23,
       87, 13, 36, 53, 45, 87, 38, 53, 33, 44,  9, 77, 25, 64, 23,  8, 73,
       83, 16, 68, 34, 16, 43, 24, 13, 30, 86, 63, 79, 77, 85, 86,  5, 30])

In [389]:
dist = tfp.distributions.OneHotCategorical(probs=prediction)
sample = tfp.distributions.Sample(dist)
sample
sess = tf.Session()
vals = sess.run(sampled_indices)

In [472]:
print(result)

M:6/8
K:G
G2A |"G"B2B BAG|"B"A2A AGF|"C"G2G GAB|"G"D3 G2F|"C"E2E EFG|
"G"D2D DGA|"A"BcB BAG|"D"A3 G2A|"G"B2B BAG|"B"A2A AGF|
"C"G2G GAB|"G"D3 G2F|"C"E2E EFG|"G"D2D DG~?XV}Luj?|zj9gGRvR:%7h
^7~_Q||3f0i##Xu9s>uB4=aOqN%r~P%FFFR]Q!eS4mh9,&H:/MUQ]Vd?[4bme0gD:_3='^>
++R'u7_FUT3ND?|"b,J-5:cTT!I[f

J8m
!)CHUa_7MqH&xvNJ/It0?e/O=!_
eiJa|M&yOJgozSJv?C3GW]ev
!hyne(OKwF1|G
|s
